# SpaceX Data Wrangling
## See end of this notebook

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Helper functions 

In [2]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

In [3]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

In [4]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [5]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

In [6]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [7]:
response = requests.get(spacex_url)

In [32]:
#print(response.content)

In [9]:
static_json_url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json'

In [10]:
response.status_code

200

In [33]:
data = response.json()
#data

In [12]:
df = pd.json_normalize(data)

In [13]:
data = df

In [14]:
# Subset of our dataframe to keep only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# Remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# Convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [15]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [16]:
BoosterVersion

[]

In [17]:
# Call getBoosterVersion
getBoosterVersion(data)

In [18]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

In [19]:
# Call getLaunchSite
getLaunchSite(data)

In [20]:
# Call getPayloadData
getPayloadData(data)

In [21]:
# Call getCoreData
getCoreData(data)

In [22]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [23]:
df = pd.DataFrame.from_dict(launch_dict, orient='index')

In [24]:
# Strangely, I had to transpose
df = df.T
#df.head()

### Filter of dataframe to only include `Falcon 9` launches


In [25]:
data_falcon9 = df.loc[df.BoosterVersion !='Falcon 1']
data_falcon9

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,6,2010-06-04,Falcon 9,None,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B0003,-80.5774,28.5619
5,8,2012-05-22,Falcon 9,525,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B0005,-80.5774,28.5619
6,10,2013-03-01,Falcon 9,677,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B0007,-80.5774,28.5619
7,11,2013-09-29,Falcon 9,500,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1,0,B1003,-120.611,34.6321
8,12,2013-12-03,Falcon 9,3170,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B1004,-80.5774,28.5619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,102,2020-09-03,Falcon 9,15600,VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5,12,B1060,-80.604,28.6081
90,103,2020-10-06,Falcon 9,15600,VLEO,KSC LC 39A,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5,13,B1058,-80.604,28.6081
91,104,2020-10-18,Falcon 9,15600,VLEO,KSC LC 39A,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5,12,B1051,-80.604,28.6081
92,105,2020-10-24,Falcon 9,15600,VLEO,CCSFS SLC 40,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5,12,B1060,-80.5774,28.5619


In [26]:
# Reset the FlgihtNumber column
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
#data_falcon9

/Applications/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


# Data Wrangling


Missing values will be replace by mean


In [27]:
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

In [28]:
df.PayloadMass.mean()

5919.16534090909

In [29]:
df.PayloadMass = df.PayloadMass.replace({np.nan:df.PayloadMass.mean()})
df

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.000000,LEO,Kwajalein Atoll,None None,1,False,False,False,None,None,0,Merlin1A,167.743,9.04772
1,2,2007-03-21,Falcon 1,5919.165341,LEO,Kwajalein Atoll,None None,1,False,False,False,None,None,0,Merlin2A,167.743,9.04772
2,4,2008-09-28,Falcon 1,165.000000,LEO,Kwajalein Atoll,None None,1,False,False,False,None,None,0,Merlin2C,167.743,9.04772
3,5,2009-07-13,Falcon 1,200.000000,LEO,Kwajalein Atoll,None None,1,False,False,False,None,None,0,Merlin3C,167.743,9.04772
4,6,2010-06-04,Falcon 9,5919.165341,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1,0,B0003,-80.5774,28.5619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,102,2020-09-03,Falcon 9,15600.000000,VLEO,KSC LC 39A,True ASDS,2,True,True,True,5e9e3032383ecb6bb234e7ca,5,12,B1060,-80.604,28.6081
90,103,2020-10-06,Falcon 9,15600.000000,VLEO,KSC LC 39A,True ASDS,3,True,True,True,5e9e3032383ecb6bb234e7ca,5,13,B1058,-80.604,28.6081
91,104,2020-10-18,Falcon 9,15600.000000,VLEO,KSC LC 39A,True ASDS,6,True,True,True,5e9e3032383ecb6bb234e7ca,5,12,B1051,-80.604,28.6081
92,105,2020-10-24,Falcon 9,15600.000000,VLEO,CCSFS SLC 40,True ASDS,3,True,True,True,5e9e3033383ecbb9e534e7cc,5,12,B1060,-80.5774,28.5619


In [30]:
##
df.PayloadMass

0        20.000000
1      5919.165341
2       165.000000
3       200.000000
4      5919.165341
          ...     
89    15600.000000
90    15600.000000
91    15600.000000
92    15600.000000
93     3681.000000
Name: PayloadMass, Length: 94, dtype: float64

In [31]:
# Verification 
df.PayloadMass.isnull().sum()

0